# Event Camera Simulator

This script allows to convert a RGB video from a traditional camera to an event video. It also has an option to convert an event video to an events list. It is also possible to convert an event list to an event video.

In [1]:
import sys
import cv2
import numpy as np
import simulator_utils as camera
sys.path.insert(0, '/Users/omkarpatil/Documents/event_based_visual_microphone/event_camera/')
import utility_spectrogram as us
import video_frames as vf

<span style="color:red"> Provide the RGB video file path bellow: </span>

In [ ]:
file_path = '/Volumes/Omkar 5T/video_dataset/chips1.avi'

This code extracts the video frames using a utiltiy function.

In [ ]:
frames = vf.extract_frames(file_path)

#### **RGB Frames to Event Frames**
This methos converts an RGB video frame from a conventional framing camera to event frames. 
- Convert RGB frame to a grayscale frame.
- Converts grayscale frame to linear log frame.
- Applies a low-pass filter based on intensity of frame to reproduce the band-pass filter nature of individual pixel circuit.
- The low-pass frame is compared with the previous memorised frame to compute an event frame.

In [12]:
video_fps = 30
sampling_period = 1/video_fps
cutoff_freq = 300
pos_thresh, neg_thresh = 0.6, 0.6
pos_frames, neg_frames, evts_video = camera.event_video(frames, sampling_period, cutoff_freq, pos_thresh, neg_thresh)

Hello!
Min difference any pixel: 0.0
Max difference any pixel: 0.0
Max events any pixel: 0
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min d

In [13]:
def save_video_from_tensor(tensor, output_path, fps=30):
    num_frames, height, width = tensor.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height), False)
    for i in range(num_frames):
        frame = tensor[i]
        out.write(frame)
    out.release()

output_path = 'moving_square_tensor.mp4'
save_video_from_tensor(evts_video, output_path)
print(f"Video saved at {output_path}")

Video saved at moving_square_tensor.mp4


#### **Event Frames to Event List**
Converts an RGB frame into an events list, which is $N\times4$ matrtix. The columns of the matrix are:
- Column 1: Time of event.
- Column 2: $x$ address of triggered pixel in frame.
- Column 3: $y$ address of triggered pixel in frame.
- Column 4: Polarity of triggered pixel in frame.

In [4]:
event_list = camera.event_list(pos_frames, neg_frames, sampling_period)

#### **Event List to Event Frames**
Converts an events list back to event frames by producing a histogram and normalising it.

In [5]:
frames = camera.events_list_frames(event_list, 512, 512, sampling_period)